In [17]:
from pandas import read_csv, to_datetime, pivot_table
import string 

## Lecture du fichier combiné de données de vente 

In [18]:
## penser à modifier le dossier d'accès au fichier
dfTot = read_csv("data Combined Buffer\\combined_sales_2.csv")

## On va rajouter une fonction qui va anonymiser les données 

In [19]:
def index_to_letter(index):
    """Convertit un index en lettre, gestion des cas où l'index est supérieur à 25 (a à z)"""
    letters = string.ascii_uppercase
    result = []
    while index >= 0:
        result.append(letters[index % 26])
        index = index // 26 - 1
    return ''.join(reversed(result))

def anon_df(df,cols):
    """
    Retourne un dataframe avec un ensemble de colonnes anonymisées 
    Input: 
    df : le dataframe en entrée
    cols : la liste de colonne qu'on souhaite rendre anonyme 
    
    """
    # On va parcourir les colonnes de la liste de colonne cols voulue 
    # On lui affecte ensuite un numéro 
    
    for col_name in cols:
        keys = {cats: index_to_letter(index = i) for i, cats in enumerate(df[col_name].unique()) }
        df[col_name] = df[col_name].apply(lambda x:keys[x])
    print(keys)
    return df

## On retire les éléments après conseil par l'expert 

## On ne prend que la F08 car on ne regarde que des objets qui sont comparables 
## Les entités F01 -> F07 ne sont pas prises car elles ne sont plus d'actualité dans les filiales de DB

## On prend le H4_GP03

In [20]:

dfTot = dfTot[dfTot["Product Family"] != "#AmbiguousMemberID - 900008"]
dfTot = dfTot[dfTot["Product Family"] != "#AmbiguousMemberID - 900009"]

dfTot = dfTot[dfTot["ENTITY"] == "F08"]

dfTot['Product Family'] = dfTot['Product Family'].replace({
    'F_DEL_MINE': 'F_DEL',
    'F_DEL_SEISM': 'F_DEL'
})

dfTot.loc[:,'UniqueID'] = dfTot['CUSTOMER GROUP ID'].astype(str) + '|' + dfTot['PRODUCT'].astype(str) +'|'+dfTot['date_dt'].astype(str)

## Conversion de Australia 

dfTot["CONTINENT"] = dfTot["CONTINENT"].str.replace("AUSTRALIA/OCEANIA","AUSTRALIA_OCEANIA")


## On va rendre anonyme le jeu de données 

In [ ]:
cols = ["Product Family"]
dfTot = anon_df(df = dfTot,cols = cols)

## On crée le fichier pour l'analyse exploratoire 

In [23]:
dfextrait = dfTot[['date_dt', 'INDICATORS', 'SIGNEDDATA', 'PRODUCT','CUSTOMER GROUP ID','UniqueID',"Product Family"]]
dfextraitqte = dfextrait[dfextrait["INDICATORS"] == "QTE"]
df_pivotqte =  dfextraitqte.pivot_table(index='Product Family', columns=['date_dt'], values='SIGNEDDATA', aggfunc='sum')

df_pivotqte.fillna(0, inplace=True)

df_pivotqte.to_csv("Product families over time.csv",index="True")


## On crée les extractions de fichiers pour le pivot table

In [24]:
dfextrait = dfTot[['date_dt', 'INDICATORS', 'SIGNEDDATA', 'PRODUCT','CUSTOMER GROUP ID','UniqueID',"Product Family"]]
dfextrait2 = dfextrait[dfextrait["Product Family"].isin(['F_DTC', 'F_DT5', 'F_DTC_BTY', 'F_DNE', 'F_DEL'])]
dfextrait3 = dfTot[['date_dt', 'INDICATORS', 'SIGNEDDATA', 'PRODUCT','CUSTOMER GROUP ID','UniqueID',"CONTINENT"]] 
dfextrait4 = dfTot[['date_dt', 'INDICATORS', 'SIGNEDDATA', 'PRODUCT','CUSTOMER GROUP ID','UniqueID',"SECTEUR"]]


## Fonction qui permet de découper les données de ventes en plusieurs fichiers csv par catégorie

In [25]:
def scinde_DF(input_df, wanted_col, index_column, columns, values_column, output_dir,aggfunc='sum'):

    dfinput = input_df
    for el in dfinput[wanted_col].unique():
        df_filtered = dfinput[dfinput[wanted_col] == el]
        
        # Créer le pivot table
        df_trait = pivot_table(df_filtered, index=index_column, columns=columns ,values=values_column, aggfunc=aggfunc)
        df_trait.reset_index(inplace=True)

        df_trait['date'] = df_trait[index_column].str.split("|").str[-1]
        df_trait['date'] = to_datetime(df_trait['date'], format='%Y-%m-%d', exact=True)

        df_trait = df_trait.drop(columns=index_column)
        df_trait = df_trait.fillna(value=0)

        # Enregistrer le pivot table au format CSV
        output_file = f"{output_dir}{el}.csv"
        df_trait.to_csv(output_file)
        print(f"Pivot table for '{el}' saved to '{output_file}'")

## On génère les fichiers pour chaque catégorie 

In [27]:
scinde_DF(input_df = dfextrait2, wanted_col = "Product Family", index_column = "UniqueID", columns = "INDICATORS", values_column = "SIGNEDDATA", output_dir='data Famille Spec\\')

In [ ]:
scinde_DF(input_df = dfextrait, wanted_col = "Product Family", index_column = "UniqueID", columns = "INDICATORS", values_column = "SIGNEDDATA", output_dir='data Prod fam tot\\')